import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=1751
cls_num=4
btch=80
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=EthanolLevel0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (504, 1752)
first_test_shape= (500, 1752)
classes_quantity= 4
tr_lbls=	 {1, 2, 3, 4}
Count_labels= [126 126 126 126]
max(train_feature_Altitude)= 4.0
min(train_feature_Altitude)= -0.99315624
first_train_sample=
 [ 1.         -0.94066444 -0.94086231 ... -0.9264013  -0.92721665
 -0.92742172]
1 126	2 126	3 126	4 126	

In [3]:
np.shape(ecg)

(504, 1752)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 126	2 126	3 126	4 126	
 max = 126


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (126, 1752)
Up to class  2 train shape =  (252, 1752)
Up to class  3 train shape =  (378, 1752)
Up to class  4 train shape =  (504, 1752)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[  0 126 126 126 126]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  2.0126435999999996
min magnitude class 1  =  -0.9870482
after normalizing >>
max magnitude class 1  =  0.9591200363364702
min magnitude class 1  =  -0.996018902061441

 cls 2  >> 
max magnitude class 2  =  2.0131951000000003
min magnitude class 2  =  -0.9829150500000007
after normalizing >>
max magnitude class 2  =  0.9594794929728245
min magnitude class 2  =  -0.9933249978065979

 cls 3  >> 
max magnitude class 3  =  2.0126618000000005
min magnitude class 3  =  -0.9908758300000002
after normalizing >>
max magnitude class 3  =  0.9591318987313604
min magnitude class 3  =  -0.9985136745093239

 cls 4  >> 
max magnitude class 4  =  2.0080999
min magnitude class 4  =  -0.9925059200000004
after normalizing >>
max magnitude class 4  =  0.9561585438276738
min magnitude class 4  =  -0.999576134470075
each class and its windows =
1 126	2 124	3 126	4 124	
 max instance in classes of test windows = 126


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.999576134470075
max tst = 0.9594794929728245


In [14]:
len(wndws_test)

500

In [15]:
np.shape(wndws_test)

(500, 1752)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(126, 1752)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (504, 1751)
y_train =>  (504,)
X_test  =>  (500, 1751)
y_test  =>  (500,)
X_valid  =>  (504, 1751)
y_valid  =>  (504,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
7/7 [==============================] - 3s 100ms/step - loss: 5.2941 - accuracy: 0.2321 - val_loss: 1.6852 - val_accuracy: 0.2500
Epoch 2/1000
7/7 [==============================] - 0s 44ms/step - loss: 1.4786 - accuracy: 0.2857 - val_loss: 1.6093 - val_accuracy: 0.2500
Epoch 3/1000
7/7 [==============================] - 0s 45ms/step - loss: 1.4578 - accuracy: 0.2381 - val_loss: 1.5656 - val_accuracy: 0.2500
Epoch 4/1000
7/7 [==============================] - 0s 44ms/step - loss: 1.4023 - accuracy: 0.2540 - val_loss: 1.5053 - val_accuracy: 0.2500
Epoch 5/1000
7/7 [==============================] - 0s 45ms/step - loss: 1.4169 - accuracy: 0.2321 - val_loss: 1.4811 - val_accuracy: 0.2500
Epoch 6/1000
7/7 [==============================] - 0s 44ms/step - loss: 1.4310 - accuracy: 0.2500 - val_loss: 1.4661 - val_accuracy: 0.2500
Epoch 7/1000
7/7 [==============================] - 0s 44ms/step - loss: 1.4140 - accuracy: 0.2897 - val_loss: 1.4536 - val_accuracy: 0.2500
Epoch 8/1000

Epoch 59/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.5372 - accuracy: 0.7718 - val_loss: 0.9103 - val_accuracy: 0.6369
Epoch 60/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.6616 - accuracy: 0.7440 - val_loss: 0.8596 - val_accuracy: 0.6528
Epoch 61/1000
7/7 [==============================] - 0s 45ms/step - loss: 0.4879 - accuracy: 0.7956 - val_loss: 0.9109 - val_accuracy: 0.5774
Epoch 62/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.7040 - accuracy: 0.7480 - val_loss: 0.7696 - val_accuracy: 0.6726
Epoch 63/1000
7/7 [==============================] - 0s 44ms/step - loss: 0.4415 - accuracy: 0.8274 - val_loss: 0.9132 - val_accuracy: 0.6091
Epoch 64/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.4223 - accuracy: 0.8413 - val_loss: 0.5808 - val_accuracy: 0.7480
Epoch 65/1000
7/7 [==============================] - 0s 44ms/step - loss: 0.3919 - accuracy: 0.8591 - val_loss: 0.7539 - val_accuracy: 0.6825
Epoch 

7/7 [==============================] - 0s 45ms/step - loss: 0.5541 - accuracy: 0.8591 - val_loss: 0.1472 - val_accuracy: 0.9563
Epoch 117/1000
7/7 [==============================] - 0s 45ms/step - loss: 0.0622 - accuracy: 0.9702 - val_loss: 0.0702 - val_accuracy: 0.9821
Epoch 118/1000
7/7 [==============================] - 0s 44ms/step - loss: 0.0448 - accuracy: 0.9802 - val_loss: 0.0251 - val_accuracy: 0.9940
Epoch 119/1000
7/7 [==============================] - 0s 44ms/step - loss: 0.0186 - accuracy: 0.9980 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 120/1000
7/7 [==============================] - 0s 44ms/step - loss: 0.0027 - accuracy: 0.9980 - val_loss: 7.2136e-04 - val_accuracy: 1.0000
Epoch 121/1000
7/7 [==============================] - 0s 44ms/step - loss: 5.9240e-04 - accuracy: 1.0000 - val_loss: 3.0062e-04 - val_accuracy: 1.0000
Epoch 122/1000
7/7 [==============================] - 0s 44ms/step - loss: 3.1837e-04 - accuracy: 1.0000 - val_loss: 1.6627e-04 - val_accuracy: 1

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

16/16 [==============================] - 0s 7ms/step
array([[30, 25, 38, 33],
       [29, 30, 29, 36],
       [32, 23, 26, 45],
       [27, 22, 36, 39]], dtype=int64)
              precision    recall  f1-score   support

           2       0.25      0.24      0.25       126
           3       0.30      0.24      0.27       124
           4       0.20      0.21      0.20       126
           5       0.25      0.31      0.28       124

    accuracy                           0.25       500
   macro avg       0.25      0.25      0.25       500
weighted avg       0.25      0.25      0.25       500



In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
7/7 [==============================] - 2s 136ms/step - loss: 23.1026 - accuracy: 0.2361 - val_loss: 1.7285 - val_accuracy: 0.2500
Epoch 2/1000
7/7 [==============================] - 1s 74ms/step - loss: 1.5287 - accuracy: 0.2440 - val_loss: 1.6936 - val_accuracy: 0.2500
Epoch 3/1000
7/7 [==============================] - 1s 75ms/step - loss: 1.4118 - accuracy: 0.2560 - val_loss: 1.6490 - val_accuracy: 0.2500
Epoch 4/1000
7/7 [==============================] - 1s 75ms/step - loss: 1.4287 - accuracy: 0.2302 - val_loss: 1.5970 - val_accuracy: 0.2500
Epoch 5/1000
7/7 [==============================] - 1s 75ms/step - loss: 1.4180 - accuracy: 0.2440 - val_loss: 1.5364 - val_accuracy: 0.2500
Epoch 6/1000
7/7 [==============================] - 1s 75ms/step - loss: 1.3963 - accuracy: 0.2679 - val_loss: 1.4945 - val_accuracy: 0.2500
Epoch 7/1000
7/7 [==============================] - 1s 74ms/step - loss: 1.4038 - accuracy: 0.2698 - val_loss: 1.4642 - val_accuracy: 0.2500
Epoch 8/100

Epoch 59/1000
7/7 [==============================] - 1s 81ms/step - loss: 1.0851 - accuracy: 0.4901 - val_loss: 1.1971 - val_accuracy: 0.4663
Epoch 60/1000
7/7 [==============================] - 1s 81ms/step - loss: 1.0442 - accuracy: 0.5258 - val_loss: 1.0700 - val_accuracy: 0.5000
Epoch 61/1000
7/7 [==============================] - 1s 74ms/step - loss: 1.0508 - accuracy: 0.5040 - val_loss: 1.1075 - val_accuracy: 0.4841
Epoch 62/1000
7/7 [==============================] - 1s 74ms/step - loss: 1.0106 - accuracy: 0.5417 - val_loss: 1.1621 - val_accuracy: 0.4782
Epoch 63/1000
7/7 [==============================] - 1s 81ms/step - loss: 0.9439 - accuracy: 0.5754 - val_loss: 1.0496 - val_accuracy: 0.5595
Epoch 64/1000
7/7 [==============================] - 1s 74ms/step - loss: 1.0255 - accuracy: 0.5397 - val_loss: 1.2083 - val_accuracy: 0.4444
Epoch 65/1000
7/7 [==============================] - 1s 81ms/step - loss: 0.9349 - accuracy: 0.5933 - val_loss: 0.9111 - val_accuracy: 0.5952
Epoch 

7/7 [==============================] - 1s 75ms/step - loss: 0.2611 - accuracy: 0.9325 - val_loss: 0.6295 - val_accuracy: 0.8254
Epoch 117/1000
7/7 [==============================] - 1s 74ms/step - loss: 0.1175 - accuracy: 0.9643 - val_loss: 0.2911 - val_accuracy: 0.9167
Epoch 118/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0607 - accuracy: 0.9802 - val_loss: 0.2901 - val_accuracy: 0.9286
Epoch 119/1000
7/7 [==============================] - 1s 74ms/step - loss: 0.3365 - accuracy: 0.9008 - val_loss: 0.1461 - val_accuracy: 0.9524
Epoch 120/1000
7/7 [==============================] - 1s 74ms/step - loss: 0.1524 - accuracy: 0.9365 - val_loss: 0.0752 - val_accuracy: 0.9683
Epoch 121/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.1432 - accuracy: 0.9563 - val_loss: 0.2046 - val_accuracy: 0.9484
Epoch 122/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0717 - accuracy: 0.9802 - val_loss: 1.5476 - val_accuracy: 0.7619
Epoch 123/1000

7/7 [==============================] - 1s 75ms/step - loss: 0.1263 - accuracy: 0.9544 - val_loss: 0.0079 - val_accuracy: 0.9980
Epoch 174/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0632 - accuracy: 0.9901 - val_loss: 0.0402 - val_accuracy: 0.9940
Epoch 175/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0279 - accuracy: 0.9980 - val_loss: 0.0220 - val_accuracy: 0.9921
Epoch 176/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0045 - accuracy: 0.9960 - val_loss: 0.0193 - val_accuracy: 0.9960
Epoch 177/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0172 - accuracy: 0.9960 - val_loss: 0.0105 - val_accuracy: 0.9960
Epoch 178/1000
7/7 [==============================] - 1s 74ms/step - loss: 0.0056 - accuracy: 0.9960 - val_loss: 0.0120 - val_accuracy: 0.9960
Epoch 179/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0111 - accuracy: 0.9960 - val_loss: 0.0112 - val_accuracy: 0.9960
Epoch 180/1000

Epoch 230/1000
7/7 [==============================] - 1s 74ms/step - loss: 0.5951 - accuracy: 0.8532 - val_loss: 0.3839 - val_accuracy: 0.8690
Epoch 231/1000
7/7 [==============================] - 1s 74ms/step - loss: 0.1379 - accuracy: 0.9563 - val_loss: 0.0088 - val_accuracy: 0.9960
Epoch 232/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0060 - accuracy: 0.9980 - val_loss: 0.0313 - val_accuracy: 0.9940
Epoch 233/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0191 - accuracy: 0.9960 - val_loss: 0.0874 - val_accuracy: 0.9762
Epoch 234/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.1602 - accuracy: 0.9643 - val_loss: 0.0905 - val_accuracy: 0.9722
Epoch 235/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0252 - accuracy: 0.9901 - val_loss: 0.0406 - val_accuracy: 0.9921
Epoch 236/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.0711 - accuracy: 0.9921 - val_loss: 0.6800 - val_accuracy: 0.7817

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

16/16 [==============================] - 0s 11ms/step
array([[39, 23, 39, 25],
       [37, 30, 27, 30],
       [35, 32, 27, 32],
       [36, 24, 37, 27]], dtype=int64)
              precision    recall  f1-score   support

           2       0.27      0.31      0.29       126
           3       0.28      0.24      0.26       124
           4       0.21      0.21      0.21       126
           5       0.24      0.22      0.23       124

    accuracy                           0.25       500
   macro avg       0.25      0.25      0.25       500
weighted avg       0.25      0.25      0.25       500

